In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [5]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [6]:
Independent=dataset[['User ID','Gender_Male','Age','EstimatedSalary']]
Independent

,User ID,Gender_Male,Age,EstimatedSalary
0,15624510,1,19,19000
1,15810944,1,35,20000
2,15668575,0,26,43000
3,15603246,0,27,57000
4,15804002,1,19,76000
...,...,...,...,...
395,15691863,0,46,41000
396,15706071,1,51,23000
397,15654296,0,50,20000
398,15755018,1,36,33000


In [7]:
Dependent=dataset[['Purchased']]
Dependent

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(Independent,Dependent,test_size=0.30,random_state=0)

In [9]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [10]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import GridSearchCV
param_grid={'alpha':[1,5,10,100],'force_alpha':[True,False],'fit_prior':[True,False]}
grid=GridSearchCV(BernoulliNB(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


C:\Users\nandh\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=BernoulliNB(), n_jobs=-1,
             param_grid={'alpha': [1, 5, 10, 100], 'fit_prior': [True, False],
                         'force_alpha': [True, False]},
             scoring='f1_weighted', verbose=3)

In [11]:
y_pred=grid.predict(x_test) #(.predict->defined function)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)
print(cm)

[[60 19]
 [ 8 33]]


In [12]:
 from sklearn.metrics import classification_report
clf_report=classification_report(y_test,y_pred)
print(clf_report)

              precision    recall  f1-score   support

           0       0.88      0.76      0.82        79
           1       0.63      0.80      0.71        41

    accuracy                           0.78       120
   macro avg       0.76      0.78      0.76       120
weighted avg       0.80      0.78      0.78       120



In [13]:
re=grid.cv_results_
grid_predictions = grid.predict(x_test) 
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,y_pred,average='weighted')
print("f1_macro value for {}:".format(grid.best_params_),f1_macro)
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])
table=pd.DataFrame.from_dict(re)
table

f1_macro value for {'alpha': 1, 'fit_prior': False, 'force_alpha': True}: 0.7798880842659645


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_fit_prior,param_force_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.029686,0.008637,0.016775,0.005683,1,True,True,"{'alpha': 1, 'fit_prior': True, 'force_alpha':...",0.782971,0.678571,0.563921,0.706970,0.739035,0.694294,0.073852,9
1,0.027256,0.005711,0.010589,0.000481,1,True,False,"{'alpha': 1, 'fit_prior': True, 'force_alpha':...",0.782971,0.678571,0.563921,0.706970,0.739035,0.694294,0.073852,9
2,0.019404,0.008804,0.016467,0.004673,1,False,True,"{'alpha': 1, 'fit_prior': False, 'force_alpha'...",0.754941,0.650298,0.702624,0.736088,0.789032,0.726597,0.047303,1
3,0.017784,0.003823,0.017380,0.003913,1,False,False,"{'alpha': 1, 'fit_prior': False, 'force_alpha'...",0.754941,0.650298,0.702624,0.736088,0.789032,0.726597,0.047303,1
4,0.011304,0.000589,0.014912,0.005081,5,True,True,"{'alpha': 5, 'fit_prior': True, 'force_alpha':...",0.782971,0.678571,0.563921,0.706970,0.739035,0.694294,0.073852,9
5,0.007128,0.002899,0.009235,0.002319,5,True,False,"{'alpha': 5, 'fit_prior': True, 'force_alpha':...",0.782971,0.678571,0.563921,0.706970,0.739035,0.694294,0.073852,9
6,0.003915,0.003831,0.016752,0.004189,5,False,True,"{'alpha': 5, 'fit_prior': False, 'force_alpha'...",0.754941,0.650298,0.702624,0.736088,0.789032,0.726597,0.047303,1
7,0.003778,0.004334,0.016405,0.003791,5,False,False,"{'alpha': 5, 'fit_prior': False, 'force_alpha'...",0.754941,0.650298,0.702624,0.736088,0.789032,0.726597,0.047303,1
8,0.007944,0.001619,0.011613,0.002147,10,True,True,"{'alpha': 10, 'fit_prior': True, 'force_alpha'...",0.782971,0.677681,0.563921,0.706970,0.739035,0.694116,0.073890,13
9,0.006556,0.004721,0.014209,0.004262,10,True,False,"{'alpha': 10, 'fit_prior': True, 'force_alpha'...",0.782971,0.677681,0.563921,0.706970,0.739035,0.694116,0.073890,13
